### Problem Statement : Given the weather condition, make a decision whether kids should go out to play or not

### The underlying implementation using only Information gain as its splitting criteria.

### importing libraries

In [165]:
import pandas as pd
import numpy as np

### creating dataframe out of popular dataset

In [166]:
string = """Outlook;Temperature;Humidity;Windy;Play
Sunny;Hot;High;False;No
Sunny;Hot;High;True;No
Overcast;Hot;High;False;Yes
Rainy;Mild;High;False;Yes
Rainy;Cool;Normal;False;Yes
Rainy;Cool;Normal;True;No
Overcast;Cool;Normal;True;Yes
Sunny;Mild;High;False;No
Sunny;Cool;Normal;False;Yes
Rainy;Mild;Normal;False;Yes
Sunny;Mild;Normal;True;Yes
Overcast;Mild;High;True;Yes
Overcast;Hot;Normal;False;Yes
Rainy;Mild;High;True;No"""

df = pd.DataFrame([item.split(';') for item in string.split('\n')])
df.columns=df.iloc[0]
df.drop(df.index[0], inplace = True)

In [167]:
df

,Outlook,Temperature,Humidity,Windy,Play
1,Sunny,Hot,High,False,No
2,Sunny,Hot,High,True,No
3,Overcast,Hot,High,False,Yes
4,Rainy,Mild,High,False,Yes
5,Rainy,Cool,Normal,False,Yes
6,Rainy,Cool,Normal,True,No
7,Overcast,Cool,Normal,True,Yes
8,Sunny,Mild,High,False,No
9,Sunny,Cool,Normal,False,Yes
10,Rainy,Mild,Normal,False,Yes


# entropy_univariate = for all 'c' sum(-p*log2(p))
# entroypy_bivariate = P(c)*E(c) for all c belogs to Class
####  P(c) : either weightage calculated from given data or probability of occurance of that class
#### E(c) : Entropy in target varible when class c is chosen
##### Note : Entropy should decrease as we move down the tree, As this is a greedy approach.
#####            Tree created in a top to bottom manner using DFS algorithm.
#####            Result evaluation uses DFS with recursion to make decision.

In [168]:
class node:
    def __init__(self):
            self.type = 'node'
            self.entropy = None #calculated entropy
            self.name = None #feature name
            self.value = None #real number for continuous variable else None
            self.data = None
            self.node_list = {}
            
class leaf:
    def __init__(self):
        self.type = "leaf"
        self.name = None
        self.value = None
        self.data = None
        
            
class DecisionTreeClassifier: #based on information gain
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    # caculating entropy for the target
    def entropy_univariate(self, df, target):
        p = df[target].value_counts()/df[target].count()
        return np.sum(np.log2(p)*p*-1)
    
    # calculating entropy for independent varibale, probability of occurance of class 'c' multiplied by entropy 
    # in target variable when 'c' is chosen.(for all c belongs to an independent variable)
    def entropy_bivariate(self, df, feature, target): #categorical
        weight = df[feature].value_counts()/df[feature].count() #weightage of each class
        entropy = np.array([self.entropy_univariate(df[df[feature]==cat],target) for cat in weight.index]) # entropy of each class
        entropy_x_y = weight*entropy
        return sum(entropy_x_y)
    
    # best feature selected whichever resulted in maximum information gain.
    def select_best_feature(self, df, features, target, entropy_target):
        _list = [(col,entropy_target - self.entropy_bivariate(df,col,target)) for col in set(features)-{target}]
        _list = sorted(_list,key=lambda x:x[1]) # sorted in ascending order
        return (_list[-1][0],entropy_target - _list[-1][1]) # returing variable with maximum IG
    
    def build_tree(self, df, features, target): #ID3 algorithm, split critiria = entropy 
        if not features: return None
        entropy_target = entropy_univariate(df,target)
    if entropy_target==0: # base condition for a recursion to stop which is reaching to a leaf node.
            l = leaf()
            l.value = df[target].mode()[0]
            #l.data = df # uncomment if want to see data at each leaf node
            return l
        else:
            selected_feature = self.select_best_feature(df, set(features), target, entropy_target)
            #print("Selected Feature : %s"%selected_feature[0])
            n = node()
            n.name = selected_feature[0]
            n.entropy = selected_feature[1]
            #n.data = df # this will store data at that level as well.
            for cat in df[n.name].unique():
                #print("Cat : %s"%cat)
                n.node_list[cat] = self.build_tree(df[df[n.name]==cat],features-{n.name},target)
            return n
        
    # evaluation uses recursion to reach to the leaf node in order to make decision.
    def evaluate(self, root_node, row):
        if root_node.type == 'leaf': return root_node.value
        #print(root_node.__dict__) #to see inside of an node/leaf
        return self.evaluate(root_node.node_list[row[root_node.name]],row)
    
    def predict(self, root_node, df):
        decision = []
        for index, row in df.iterrows():
            decision.append(self.evaluate(root_node,row))
        return np.array(decision)
    
tree=DecisionTreeClassifier(df[set(df.columns)-{"Play"}],df[["Play"]])
root = tree.build_tree(df,set(df.columns)-{"Play"},"Play")


### creating data to test results

In [169]:
test_string = """Outlook;Temperature;Humidity;Windy
Rainy;Cool;Normal;True
Overcast;Cool;Normal;True
Sunny;Mild;High;False"""

test_string_actual_results = ["No","Yes","No"]
print("Actual Results : %s"%str(test_string_actual_results))
test_df = pd.DataFrame([item.split(';') for item in test_string.split('\n')])
test_df.columns=test_df.iloc[0]
test_df.drop(test_df.index[0], inplace = True)
predicted_results = tree.predict(root, test_df)
print("Predicted Results : %s"%str(predicted_results))

Actual Results : ['No', 'Yes', 'No']
Predicted Results : ['No' 'Yes' 'No']
